In [0]:
cd /content/drive/My Drive/Colab Notebooks/Tobigs_week7_assignment

/content/drive/My Drive/Colab Notebooks/Tobigs_week7_assignment


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls

cnn_submission2.csv         radam_submission.csv   train/
cnn_submission.csv          sample_submission.csv  train_df.csv
cnn_training.ipynb          submission.csv         week7_Keras_강미경.ipynb
da2_submission.csv          test_df.csv            week7_Pytorch_강미경.ipynb
data_argumentatioin2.ipynb  torch2_submission.csv
data_argumentation.ipynb    torch_submission.csv


## 전처리...

In [0]:
import pandas as pd
sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [0]:
type(train.iloc[:, 1:].values/255)
#values로 반환하면 numpy array형태로 반환되는 것 같다. 
#(그냥 반환하면 pd.DataFrame형태)

numpy.ndarray

In [0]:
X = train.iloc[:,1:].values / 255
y = train.iloc[:,0].values
X_test = test.iloc[:,1:].values / 255

In [0]:
X.shape

(42000, 784)

## Dataset이랑 Loader 정의하기

In [0]:
from torch.utils.data import Dataset, DataLoader

#Train 데이터의 로더를 정의
class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

#Test 데이터의 로더를 정의
class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        return X

In [0]:
traindataloader = DataLoader(TrainDataset(X, y), batch_size=128, shuffle=True, num_workers=4)
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
  #layer를 선언하는 부분
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512) #Keras의 Dense와 같은 역할 :: fully connected layer
        self.fc2 = nn.Linear(512, 256) #Keras와는 달리 Pytorch는 입력개수와 출력 개수 둘 다 지정해주어야 함.
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
        self.elu = nn.ELU()
        self.dropout1 = torch.nn.Dropout(p=0.5)
        self.dropout2 = torch.nn.Dropout(p=0.5)

  #activation function을 지정하는 부분
    def forward(self, x):
        x = F.relu(self.fc1(x)) #첫 번째 선형결합에 relu씌워 반환
        x = F.relu(self.fc2(x)) #두 번째 선형결합에 relu씌워 반환
        x = self.dropout1(x) #dropout으로 Regularization을 챙겨준다
        x = self.elu(self.fc3(x)) #세 번째 선형결합에 elu씌워 반환 
        x = self.dropout2(x) #droupout으로 Regularization을 챙겨준다
        x = self.fc4(x) #네 번째 선형결합 반환
        #마지막 레이어에는 activation function인 Softmax를 지정해줄 필요가 없다. 
        #왜냐 corssentropy를 해주면 알아서 Softmax까지 해주기 때문에
        return x

#weight initialization
def init_weights(m):
    if type(m) == nn.Linear:
        print("[weight init apply] : ", m)
        torch.nn.init.kaiming_uniform_(m.weight)
        m.bias.data.fill_(0.01)

net = Net()
net.apply(init_weights)

[weight init apply] :  Linear(in_features=784, out_features=512, bias=True)
[weight init apply] :  Linear(in_features=512, out_features=256, bias=True)
[weight init apply] :  Linear(in_features=256, out_features=128, bias=True)
[weight init apply] :  Linear(in_features=128, out_features=10, bias=True)


Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [0]:
net #모델 정보 확인

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [0]:
import torch.optim as optim
#Optimazation function들이 저장되어 있는 optim을 import

#Lossfunction과 Optimizaer를 지정해준다. Keras의 Compile부분.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)  #2e-3

#sgd :: 성능이 그다지 좋아지지 않았다. 더 오래오래 학습을 시켜야하는 것 같다. 하지만 나에겐 주어진 시간의 한계가 있으니 adam을 쓰기로한다.
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
#cpu를 사용할까 gpu로 사용할까?
#결과를 보니 cuda로 돌아가고 있는 걸 알 수 있다.

device(type='cuda', index=0)

In [0]:
net.to(device)
#net.to를 이용해 쉽게 모델을 gpu에 올릴 수 있다.

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [0]:
len(traindataloader)

329

In [0]:
num_epochs = 100
#train data를 이용해 학습
for epoch in range(num_epochs):  # 지정한 학습 수 만큼 loop

    running_loss = 0.0
    for batch_idx, data in enumerate(traindataloader): #traindata를 이용해 데이터를 차례대로 올림.
        inputs, labels = data #trainloader에 의해 X, y가 차례대로 inputs, labels에 반환됨.
        inputs = torch.tensor(inputs, device=device).float() #이를 tensor형태로 변환하고 gpu에 올림
        labels = torch.tensor(labels, device=device) #labels은 따로 원핫 인코딩을 해주지 않았기 때문에 0~9까지의 숫자

        # gradient를 초기화해준다.
        optimizer.zero_grad() #
        # forward + backward + optimize
        outputs = net(inputs) #모델에 inputs값을 넣어서 outputs을 출력함.
        # 이때 outputs은 one-hot-encoding이 되어있는 상태로 나옴.
        
        loss = criterion(outputs, labels)
        #위에서 지정해준 crossentropy를 이용해 loss를 계산. 이때 outputs, labels 순서로 넣는 것에 주의
        #둘의 인코딩 상태가 다르지만 똑똑한 함수가 알아서 맞춰서 계산 해준다.
        loss.backward() #loss를 가지고 back propagation을 수행하여 gradients계산
        optimizer.step() #계산된 gradient를 optimizer에 넣어 가중치 업데이트
        
        # print statistics
        running_loss += loss.item()
    print("Epoch : {} loss: {}".format(epoch, running_loss/len(traindataloader)))
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Epoch : 0 loss: 2.2160238933418297
Epoch : 1 loss: 1.6657814164292124
Epoch : 2 loss: 1.271076948265899
Epoch : 3 loss: 0.9908328087134202
Epoch : 4 loss: 0.7960695283753532
Epoch : 5 loss: 0.656201456274305
Epoch : 6 loss: 0.5693313208575669
Epoch : 7 loss: 0.49745592212242196
Epoch : 8 loss: 0.44180318686012804
Epoch : 9 loss: 0.4020582615966855
Epoch : 10 loss: 0.3668863760211185
Epoch : 11 loss: 0.33778724255532844
Epoch : 12 loss: 0.3170692762161823
Epoch : 13 loss: 0.298210079637342
Epoch : 14 loss: 0.2835782868702723
Epoch : 15 loss: 0.2683419504154779
Epoch : 16 loss: 0.2500745190539621
Epoch : 17 loss: 0.24736341238474774
Epoch : 18 loss: 0.23371777133195235
Epoch : 19 loss: 0.22522271533233418
Epoch : 20 loss: 0.21290113840331423
Epoch : 21 loss: 0.2091420592462763
Epoch : 22 loss: 0.20091506107328147
Epoch : 23 loss: 0.19330152620116992
Epoch : 24 loss: 0.18597630495177214
Epoch : 25 loss: 0.18328479949192436
Epoch : 26 loss: 0.17745353540796277
Epoch : 27 loss: 0.1704048544

In [49]:
import tensorflow as tf
torch.max(tf.constant[1, 2, 3], 1)

TypeError: ignored

In [0]:
#test set 
correct = 0 #맞은 개수
total = 0 
net.eval() #이제 평가모드로 전환함.
preds = []
with torch.no_grad(): #테스트에서는 역전파가 일어나지 않기 때문에 gradient를 하지 않음.
    for data in testdataloader:
        inputs = data #데이터로더로부터 X_test를 반환
        inputs = torch.tensor(inputs, device=device).float() #모델에 올려줌.
        outputs = net(inputs) #학습한 모델에 inputs을 넣고 outputs을 반환받음. (원핫인코딩된 형태로 반환됨)
        _, predicted = torch.max(outputs.data, 1) #행에서 가장 큰 값들만을 추려서 predicted로 반환 
        for pred in predicted: 
          preds.append(pred.cpu().numpy()) #pred를 cuda에서 cpu로 가지고와서 numpy로 변환.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [0]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [0]:
import numpy as np
preds = np.array(preds)
print(preds.shape)

(18000,)


In [0]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("torch_adam_submission.csv", index=False)